In [ ]:
HTML(read(open("style.css"), String))

# Advanced Board Struct
This notebook implements the struct `AdvBoard` which combines additional attributes the the `Board` class of the chess.jl Library. These attributes relate to the `Board` directly.

Added Attributes:
1. `score::Int64` is the static evaluation of the board in centipawns
1. `hash::UInt64` is the hash of the `Board`
1. `repCounter::Dict{UInt64, Int8}` is a dictionary counting how often a position has been reached. This is needed to check whether a 3-fold-repetition has occured resulting in a draw.

In [ ]:
using Chess
using NBInclude

In [ ]:
@nbinclude("Repetition.ipynb")

This notebook contains the `Advanced Board` (short: `AdvBoard`) struct which bundles additional information about the board to the board. As the `::Board` Class of Chess.jl does not contain all the information we need to the board this will be expanded to contain the following additional contents:

1. `score::Int64` is the static evaluation of the board in centipawn
1. `hash::UInt64`  is the hash of the board
1. `repCounter::Dict{UInt64, Int8}` is a Dictionary that counts how often a position as occured

In [ ]:
mutable struct AdvBoard
    state::Board
    score::Int64
    hash::UInt64
    repCounter::Dict{UInt64, Int8}
end

## Constructor for AdvBoard

The Constructor `AdvBoard` creates an `AdvBoard` from a `Board` and initializes all attributes automatically.

Arguments:
1. `board::Board` is the board that the AdvBoard should be representing

Returns an `AdvBoard` with calculated attributes.

In [ ]:
function AdvBoard(board::Board)::AdvBoard
    score = evaluate_position(board)
    hash  = zobrist_hash(board)
    repCounter = Dict{UInt64, Int8}()
    incrementHash!(repCounter, hash)
    return AdvBoard(board, score, hash, repCounter)
end

### function domoveAdv!

The function `domoveAdv!` has the same functionality for the `AdvBoard` as the `domove!` function for a `Board`. It applies the given move on the AdvBoard and changes all attributes according to the new position.

Arguments:
1. `aBoard::AdvBoard` is the chess board where the move should be applied
1. `move::Move` is the move that should be done

Returns a three-tuple including information needed to undo the move. These values should not be mutated.

In [ ]:
function domoveAdv!(aBoard::AdvBoard, move::Move)
    oldscore:: Int64 = aBoard.score
    oldhash:: UInt64 = aBoard.hash
    aBoard.score = evaluate_move(aBoard, move) 
    aBoard.hash = zobrist_hash(aBoard.state, aBoard.hash, move) 
    undoinfo = domove!(aBoard.state, move)
    incrementHash!(aBoard.repCounter, aBoard.hash)
    return (oldscore, oldhash, undoinfo)
end

### function undomoveAdv!

The function `undomoveAdv!` has the same functionality for the `AdvBoard` as the `undomove!` function for a `Board`. It undoes the last move doneon  the AdvBoard and changes all attributes according to the new position.

Arguments:
1. `aBoard::AdvBoard` is the chess board where the move should be applied
1. `undoinfo` is the three-tuple returned from the `domoveAdv!` function

In [ ]:
function undomoveAdv!(ABoard::AdvBoard, undoinfo)
    decrementHash!(ABoard.repCounter, ABoard.hash)
    undomove!(ABoard.state, undoinfo[3])
    ABoard.score = undoinfo[1]
    ABoard.hash  = undoinfo[2]
end